In [5]:
from LSTM_model import *
import torch
import pytreebank
import spacy
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#Load GPU
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

#Load Data
train = pytreebank.import_tree_corpus("./sentiment/train.txt")
tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

def seperate(dataset): #Use All nodes
    seperation = []
    for data in dataset:
        for label, text in data.to_labeled_lines():
            seperation.append((label,text))
    return seperation

train_sep = seperate(train)
text_pipeline = lambda x: vocab(tokenizer(x))

def yield_tokens(data_iter): #data_iter, e.g., train
    for _, text in data_iter: 
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_sep), specials=['<unk>','<pad>','<bos>','<eos>'], special_first = True)
vocab.set_default_index(vocab["<unk>"])
pad_ix = vocab['<pad>']

#hyper-parameter
input_dim  = len(vocab)
hid_dim    = 256
emb_dim    = 300         
output_dim = 5
num_layers = 2
bidirectional = True
dropout = 0.5

model = LSTM(input_dim, emb_dim, hid_dim, output_dim, num_layers, bidirectional, dropout, pad_ix).to(device)
save_path = f'models/LSTM_TreeBank.pt'
model.load_state_dict(torch.load(save_path))

<All keys matched successfully>

In [8]:
# reddit crawler
import praw
your_client_id='ElyKc6o3du1IMb5LP2HYjg'
your_client_secret='YimfVn3bTLyVFu_XkJuLxrNkR3vHAQ'
your_user_name='guntsv'

reddit = praw.Reddit(client_id=your_client_id,
                     client_secret=your_client_secret,
                     user_agent=your_user_name,
                     check_for_async=False)

In [9]:
subreddit = reddit.subreddit('iphone')
topics = [*subreddit.top(limit=None)] # top posts all time
print(len(topics))

981


In [13]:
#Check Title
for topic in topics:
    print(topic.title) # headline
    break

Now you can cook on it


In [15]:
import pandas as pd

title = [n.title for n in topics]
df_topics = pd.DataFrame({"title": title})
df_topics.head()

,title
0,Now you can cook on it
1,Just got my first iPhone for Christmas!
2,Apple CarPlay volume control UI
3,Wait I’ve seen this one before
4,How rotation lock should be


In [16]:
df_topics.shape

(981, 1)

In [ ]:
def prediction(test_str_list):
    result = list()
    for test_str in test_str_list:
        text = torch.tensor(text_pipeline(test_str)).to(device).reshape(1, -1)
        # text_list = [x.item() for x in text]
        text_length = torch.tensor([text.size(1)]).to(dtype=torch.int64)
        output = model(text, text_length).squeeze(1)
        predicted = torch.max(output.data, 1)[1] #.detach().cpu().numpy()
        result.append((test_str, predicted))
    return result

In [20]:
prediction(df_topics['title'][:5])

[('Now you can cook on it', tensor([2], device='cuda:2')),
 ('Just got my first iPhone for Christmas!', tensor([3], device='cuda:2')),
 ('Apple CarPlay volume control UI', tensor([2], device='cuda:2')),
 ('Wait I’ve seen this one before', tensor([2], device='cuda:2')),
 ('How rotation lock should be', tensor([2], device='cuda:2'))]